# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np

import random
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

import os
from collections import defaultdict
from tqdm import tqdm_notebook

from transformers import AutoConfig, AutoTokenizer, AutoModel
import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import torch.nn.init as init
import torch.nn.functional as F
from torch.nn import Parameter
from torch.autograd.function import InplaceFunction
import math

from torch.utils.data import Sampler, Dataset, DataLoader
import random

from more_itertools import chunked, flatten

In [ ]:
%cd drive/MyDrive/CommonLit

/content/drive/MyDrive/CommonLit


# Get folds

In [ ]:
df = pd.read_csv("train_folds.csv")

# Seed Everything

In [ ]:
def seed_everything(seed=12):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=12)

# Configuration

In [ ]:
MAX_LEN = 256
EPOCHS = 4
ROBERTA_PATH = "roberta-large"
PRETRAIN_PATH = "Pretrain_CLRP_Roberta/pretrained_roberta_large.bin"
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
TOKENIZER = transformers.AutoTokenizer.from_pretrained(ROBERTA_PATH)

# Dataset

In [ ]:
class RobertaDataset:
    def __init__(self,df):
        self.excerpt = df.excerpt.values
        self.target = df.target.values

    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self,item):
        excerpt = str(self.excerpt[item])
        excerpt = " ".join(excerpt.split())
        inputs = TOKENIZER(excerpt, add_special_tokens=True, max_length=MAX_LEN, padding=True, truncation=True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        
        padding_len = MAX_LEN-len(ids)
        ids = ids+([0]*padding_len)
        mask = mask+([0]*padding_len)
 
        return {"ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float)}

# Model Building

In [ ]:
class RobertaModel(nn.Module):
    
    def __init__(self, model_type="mean"):
        super(RobertaModel,self).__init__()

        self.model_type = model_type
        
        self.config = AutoConfig.from_pretrained(ROBERTA_PATH)
        self.config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 0.0})   
        self.roberta = transformers.AutoModel.from_pretrained(PRETRAIN_PATH, config=self.config)

        if model_type == "attention":
            
            self.attention = nn.Sequential(            
            nn.Linear(1024, 256),            
            nn.Tanh(),                       
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
            )   

            self.linear = (nn.Linear(1024, 1))
                           
        elif model_type == "mean":
        
            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 768)
            self.linear2 = nn.Linear(768, 1)
            self.layer_norm2 = nn.LayerNorm(768)

    def freeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = False

    def unfreeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = True
        
    def forward(self, ids, mask, loss_fn = None, targets = None):

        if self.model_type == "mean":

            outputs = self.roberta(ids, mask)
            last_hidden_state = outputs[0]
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            norm_mean_embeddings = self.layer_norm1(mean_embeddings)
            logits = self.linear1(norm_mean_embeddings)
            logits = self.linear2(self.layer_norm2(logits))

        elif self.model_type=="attention":

            roberta_output = self.roberta(input_ids=ids,
                                  attention_mask=mask)        
            last_layer_hidden_states = roberta_output.last_hidden_state
            weights = self.attention(last_layer_hidden_states)
            context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
            logits = self.linear(context_vector)

        if targets is not None:

            loss = torch.sqrt(loss_fn(logits.view(-1),targets.view(-1)))
            return loss, logits

        else:

            return logits

# Evaluation Scheduler

In [ ]:
def evaluate(EVAL_STEPS,valid_interval, valid_loss,train_loss, final_train_loss, index, best_loss, epoch):
    
        print(f"Epoch:{epoch}| Batch {index} | Train Loss:{train_loss.avg()} | Validation loss:{valid_loss}")
        if (valid_loss < best_loss):
            
            for rmse, steps in EVAL_STEPS:
                if valid_loss > rmse:
                    valid_interval = steps
                    break
              
            print(f"Validation loss decreased from {best_loss} to {valid_loss}.")
            final_train_loss = train_loss.avg()
            best_loss = valid_loss
            torch.save(model.state_dict(),f'Models/CodeRobertaLargeNoEps/model{fold}.bin')

            
        return valid_interval, best_loss, final_train_loss

# Training Function

In [ ]:
def train_fn(train_dataloader, valid_dataloader, model, optimizer, device, scheduler):
    

    EVAL_STEPS = [(0.50,80),(0.49,60), (0.48, 40), (-1., 40)]
    valid_interval = EVAL_STEPS[0][1]
    best_loss = np.inf
    final_train_loss = None
    accumulation_steps = 4
    lr_schedule = [5e-5, 2e-5, 5e-6, 2e-6]
    # lr_schedule = [4e-5, 2e-5, 1e-5, 5e-6,2.5e-6]

    for epoch in range(EPOCHS):
      
        train_loss = AvgCounter()
        lr = lr_schedule[epoch]
        optimizer = scheduler(optimizer,lr)

        for index, d in tqdm_notebook(enumerate(train_dataloader), total=len(train_dataloader)):
  
            ids = d["ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            model.train()
            loss, outputs = model(ids=ids, mask=mask, loss_fn = loss_fn, targets = targets)
            
            train_loss.update(loss.item(), len(d))
            loss = loss / accumulation_steps 
            loss.backward()

            if index % accumulation_steps == 0:             
                optimizer.step() 
                # scheduler.step()                           
                optimizer.zero_grad()

            if (index % valid_interval == 0) | ((len(train_dataloader)-index) == 1):

                valid_loss = eval_fn(valid_dataloader,model,device)
                
                valid_interval, best_loss, final_train_loss = evaluate(EVAL_STEPS,valid_interval, valid_loss,train_loss, final_train_loss, index, best_loss, epoch )
            
    return final_train_loss, best_loss

# Evaluation Function

In [ ]:
def eval_fn(data_loader, model, device):
    model.eval()
    valid_loss = AvgCounter()
    
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            loss, outputs = model(ids=ids, mask=mask, loss_fn = loss_fn, targets = targets)
            
            valid_loss.update(loss.item(), len(d))
            
    return valid_loss.avg()

# AvgCounter

In [ ]:
class AvgCounter:
    def __init__(self):
        self.reset()
        
    def update(self, loss, n_samples):
        self.loss += loss * n_samples
        self.n_samples += n_samples
        
    def avg(self):
        return self.loss / self.n_samples
    
    def reset(self):
        self.loss = 0
        self.n_samples = 0

# Useful Functions

In [ ]:
# create pytorch dataloader
def create_dataloader(df, fold, smart = True):
    
    train = df[df.kfold!=fold].reset_index(drop=True)
    valid = df[df.kfold==fold].reset_index(drop=True)

    if smart:
        train_dataset = RobertaDatasetSmart(train)
        valid_dataset = RobertaDatasetSmart(valid)

        sampler_train = SmartBatchingSampler(train_dataset, TRAIN_BATCH_SIZE)
        sampler_valid = SmartBatchingSampler(valid_dataset, VALID_BATCH_SIZE)

        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= TRAIN_BATCH_SIZE, sampler = sampler_train, collate_fn=pad_collator)
        valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size= VALID_BATCH_SIZE, sampler = sampler_valid, collate_fn=pad_collator)

    else:
        train_dataset = RobertaDataset(train)
        valid_dataset = RobertaDataset(valid)
    
        sampler = torch.utils.data.RandomSampler(train_dataset)

        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= TRAIN_BATCH_SIZE, sampler = sampler  )
        valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size= VALID_BATCH_SIZE)

    return train_dataloader, valid_dataloader

In [ ]:
def create_model(device):

    model = RobertaModel().to(device)
    
    return model

In [ ]:
# create the optimizer
def create_optimizer(model):
    named_parameters = list(model.named_parameters()) 
    no_decay = ['bias', 'gamma', 'beta']   
    
    parameters = []
    lr = 3e-5
    regressor_lr = 2e-5
    for layer in range(23,-1,-1):
        layer_params = {
          'params': [
                      p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay) \
                      and (f'encoder.layer.{layer}.' in n)
                      ],
          'lr': lr
      }
        parameters.append(layer_params)

        lr *= 0.975

    regressor_params = {
      'params': [p for n,p in model.named_parameters() if "roberta" not in n],
      'lr': regressor_lr
    }

    parameters.append(regressor_params)

    regressor_params = {
      'params': [
                      p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay) \
                      and (f'roberta.embeddings' in n)
                      ],
      'lr': regressor_lr
    }
    parameters.append(regressor_params)

    return AdamW(parameters)

In [ ]:
# create scheduler
def create_scheduler(optimizer, num_warmup_steps, num_train_steps, scheduler_name = "get_cosine_schedule_with_warmup" ):

    if scheduler_name == "get_linear_schedule_with_warmup":
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
        
    elif scheduler_name == "get_cosine_schedule_with_warmup":
        scheduler = get_cosine_schedule_with_warmup(optimizer,num_training_steps=num_train_steps,num_warmup_steps=50) 
        
    else:
        raise Exception(f"Unknown scheduler: {scheduler_name}")

    return scheduler

In [ ]:
def scheduler(optimizer,lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

# Training

In [ ]:
loss_fn=nn.MSELoss()
loss=defaultdict(list)
results_val = {}
results_train = {}
for fold in range(5):

    seed_everything(12)
    
    device = torch.device("cuda")
    model = create_model(device)

    print("################################")
    print(f"Training Fold {fold}")
    print("################################")

    train_dataloader, valid_dataloader = create_dataloader(df, fold, smart = False)
    num_train_steps = len(train_dataloader) * EPOCHS

    optimizer = create_optimizer(model)
    # scheduler = create_scheduler(optimizer, num_warmup_steps = 0, num_train_steps = num_train_steps )

    seed_everything(12)
    
    results_train[fold], results_val[fold] = train_fn(train_dataloader,valid_dataloader, model, optimizer, device, scheduler)

print("################################")
print("RESULTS")
print("################################")
cv_val = np.mean([results_val[i] for i in range(5)])
cv_train = np.mean([results_train[i] for i in range(5)])
print(f"Results of cross validation for seed 12: Train : {cv_train}, Val : {cv_val}") # I always used seed 12

Some weights of the model checkpoint at Pretrain_CLRP_Roberta/pretrained_roberta_large.bin were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at Pretrain_CLRP_Roberta/pretrained_roberta_large.bin and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probab

################################
Training Fold 0
################################


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Epoch:0| Batch 0 | Train Loss:0.7076603174209595 | Validation loss:1.6521922467460095
Validation loss decreased from inf to 1.6521922467460095.
Epoch:0| Batch 80 | Train Loss:0.8113821549915973 | Validation loss:0.7384189463837046
Validation loss decreased from 1.6521922467460095 to 0.7384189463837046.
Epoch:0| Batch 160 | Train Loss:0.7309419739505519 | Validation loss:0.6017998343622181
Validation loss decreased from 0.7384189463837046 to 0.6017998343622181.
Epoch:0| Batch 240 | Train Loss:0.6824526216108274 | Validation loss:0.5486489819808745
Validation loss decreased from 0.6017998343622181 to 0.5486489819808745.
Epoch:0| Batch 283 | Train Loss:0.6669465970300453 | Validation loss:0.5277437735611284
Validation loss decreased from 0.5486489819808745 to 0.5277437735611284.



Epoch:1| Batch 0 | Train Loss:0.4974327087402344 | Validation loss:0.5049597679729193
Validation loss decreased from 0.5277437735611284 to 0.5049597679729193.
Epoch:1| Batch 80 | Train Loss:0.3879103347843076 | Validation loss:0.5332624364486882
Epoch:1| Batch 160 | Train Loss:0.3824170788993006 | Validation loss:0.48577200023221295
Validation loss decreased from 0.5049597679729193 to 0.48577200023221295.
Epoch:1| Batch 200 | Train Loss:0.38069388359340267 | Validation loss:0.5066926789955354
Epoch:1| Batch 240 | Train Loss:0.3815836522465425 | Validation loss:0.49179812573211296
Epoch:1| Batch 280 | Train Loss:0.3843456796267703 | Validation loss:0.5126626128461998
Epoch:1| Batch 283 | Train Loss:0.3845754325599738 | Validation loss:0.5126626128461998



...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Epoch:0| Batch 0 | Train Loss:1.2930927276611328 | Validation loss:1.265312889931907
Validation loss decreased from inf to 1.265312889931907.
Epoch:0| Batch 80 | Train Loss:0.8215533207964014 | Validation loss:0.7354658535668548
Validation loss decreased from 1.265312889931907 to 0.7354658535668548.
Epoch:0| Batch 160 | Train Loss:0.7347561787374272 | Validation loss:0.7050184624295839
Validation loss decreased from 0.7354658535668548 to 0.7050184624295839.
Epoch:0| Batch 240 | Train Loss:0.686449088125308 | Validation loss:0.5525530329052831
Validation loss decreased from 0.7050184624295839 to 0.5525530329052831.
Epoch:0| Batch 283 | Train Loss:0.6675378975104278 | Validation loss:0.5468806840584312
Validation loss decreased from 0.5525530329052831 to 0.5468806840584312.



Epoch:1| Batch 0 | Train Loss:0.5113670229911804 | Validation loss:0.5617832495293147
Epoch:1| Batch 80 | Train Loss:0.41836675725601336 | Validation loss:0.522825215064304
Validation loss decreased from 0.5468806840584312 to 0.522825215064304.
Epoch:1| Batch 160 | Train Loss:0.4071887458518425 | Validation loss:0.5029745227854017
Validation loss decreased from 0.522825215064304 to 0.5029745227854017.
Epoch:1| Batch 240 | Train Loss:0.3986223547785114 | Validation loss:0.5018246112994744
Validation loss decreased from 0.5029745227854017 to 0.5018246112994744.
Epoch:1| Batch 283 | Train Loss:0.39728344121659304 | Validation loss:0.492489153230694
Validation loss decreased from 0.5018246112994744 to 0.492489153230694.



Epoch:2| Batch 0 | Train Loss:0.16038724780082703 | Validation loss:0.4913192613863609
Validation loss decreased from 0.492489153230694 to 0.4913192613863609.
Epoch:2| Batch 60 | Train Loss:0.2390093678822283 | Validation loss:0.4868868272489225
Validation loss decreased from 0.4913192613863609 to 0.4868868272489225.
Epoch:2| Batch 80 | Train Loss:0.2371081292261312 | Validation loss:0.48926026048794596
Epoch:2| Batch 120 | Train Loss:0.23879925758878062 | Validation loss:0.48390443392202886
Validation loss decreased from 0.4868868272489225 to 0.48390443392202886.
Epoch:2| Batch 160 | Train Loss:0.24325920261951708 | Validation loss:0.4859177455515929
Epoch:2| Batch 200 | Train Loss:0.23908830011513696 | Validation loss:0.4861511225011987
Epoch:2| Batch 240 | Train Loss:0.23871735804926805 | Validation loss:0.4800774509638128
Validation loss decreased from 0.48390443392202886 to 0.4800774509638128.
Epoch:2| Batch 280 | Train Loss:0.23741460962757946 | Validation loss:0.4801594770290482

Epoch:3| Batch 0 | Train Loss:0.1288483887910843 | Validation loss:0.4825835276237676
Epoch:3| Batch 40 | Train Loss:0.15192920805477514 | Validation loss:0.48153174993857534
Epoch:3| Batch 80 | Train Loss:0.1729275909838853 | Validation loss:0.4825236073262255
Epoch:3| Batch 120 | Train Loss:0.17500811713663014 | Validation loss:0.4869215662630511
Epoch:3| Batch 160 | Train Loss:0.1688942051498416 | Validation loss:0.4799799986288581
Validation loss decreased from 0.4800774509638128 to 0.4799799986288581.
Epoch:3| Batch 200 | Train Loss:0.16901029848414867 | Validation loss:0.47886615886654654
Validation loss decreased from 0.4799799986288581 to 0.47886615886654654.
Epoch:3| Batch 240 | Train Loss:0.17174281979064723 | Validation loss:0.48108336518348105
Epoch:3| Batch 280 | Train Loss:0.17128354993738312 | Validation loss:0.4774508830946936
Validation loss decreased from 0.47886615886654654 to 0.4774508830946936.
Epoch:3| Batch 283 | Train Loss:0.1715512820716265 | Validation loss:0.

In [ ]:
# Results of cross validation for seed 42: Train : 0.3348806785935321, Val : 0.48983509208954557 LB 0.489
# Results of cross validation for seed 42: Train : 0.2606250740913959, Val : 0.4757214925658535 LB 0.468
# Results of cross validation for seed 42: Train : 0.20288117347075163, Val : 0.4683537180574847 LB 0.464 Change the shape to 768
# increase accumulation steps from 4 to 8
# Results of cross validation for seed 42: Train : 0.29265708032267856, Val : 0.4810768190404059
# Same accumulation step but proper learning rate !
# Results of cross validation for seed 42: Train : 0.23795280918271736, Val : 0.4706005893122982 LB ?
# change layer norm eps 1e-2
# Results of cross validation for seed 12: Train : 0.2127049908399834, Val : 0.46800723113644294
# no layer norm in roberta
# Results of cross validation for seed 12: Train : 0.2150556268539044, Val : 0.46881619830366594